In [1]:
from flask import Flask, request
from flask_restful import Resource, Api, reqparse
from flask_jwt import JWT, jwt_required
from security import authenticate, identity

app = Flask(__name__)
app.secret_key = 'Flash'

api = Api(app)

jwt = JWT(app, authenticate, identity) #new endpoint /auth

In [2]:
items = []

In [3]:
class Item(Resource):
    
    parser = reqparse.RequestParser()
    parser.add_argument('price',
                        type=float,
                        required=True,
                        help='this field cannot be left blank'
                       )
    
    #@jwt_required()
    def get(self): #
        name = request.args.get('test') #
        score = request.args.get('score') #
        item = next(filter(lambda x: x['name'] == name, items),None)
        return {'item': name,'score': score}, 200 if item else 404 #
    
    def post(self,name):
        
        if next(filter(lambda x: x['name'] == name, items),None):
            return {'message': "An item with name '{}' already exists.".format(name)}, 400
        
        data = Item.parser.parse_args()
        
        data = request.get_json()
        item = {'name':name,'price':data['price']}
        items.append(item)
        return item, 201
    
    def delete(self,name):
        global items
        items = list(filter(lambda x: x['name'] != name, items))
        return {'message': 'Item delete'}
    
    def put(self,name):
        data = Item.parser.parse_args()
        item = next(filter(lambda x: x['name'] == name, items), None)
        if item is None:
            item = {'name': name, "price": data['price']}
            items.append(item)
        else:
            item.update(data)
        return item

In [4]:
class ItemList(Resource):
    def get(self):
        return {'items': items}, 200

In [5]:
api.add_resource(Item, '/item')
#api.add_resource(Item, '/item/<string:name>')
api.add_resource(ItemList, '/items')

In [6]:
app.run(host='0.0.0.0',port=5565)

 * Running on http://0.0.0.0:5565/ (Press CTRL+C to quit)
148.87.19.40 - - [02/Oct/2017 17:20:34] "GET /item?test=what&score=123 HTTP/1.1" 404 -
148.87.19.40 - - [02/Oct/2017 17:20:40] "GET /item?test=what?&score=123 HTTP/1.1" 404 -
148.87.19.40 - - [02/Oct/2017 17:20:47] "GET /item?test=what? HTTP/1.1" 404 -
